In [1]:
import igl
import meshplot as mp
import vedo as vd
import numpy as np
import os 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

vd.settings.default_backend = 'k3d'

## Initial Line Congruence test

### Visualization setup

In [1]:
dir_path = os.getcwd()

data_path = dir_path+"/approximation/data/"

mv, mf = igl.read_triangle_mesh(os.path.join(data_path,"centers.obj" ))
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,"test_remeshed.obj" ))
# Get vertex normals for test mesh
tn = igl.per_vertex_normals(tv, tf)

signs = np.sign(np.sum(tn * ([0,0,1]), axis=1))

# Fix normal directions
tn = tn * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Create dual mesh
mesh = Mesh()

mesh.make_mesh(tv,tf)

# Get inner vertices
inner_vertices = mesh.inner_vertices()

# Dual topology including outer faces
dual = mesh.dual_top()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]


# dual_faces = dualmesh.faces()
# # Draw vertex normals
# # vd.show(vd.Points(tv, c='red'), vd.Arrows(tv, tv + tn * 0.1, c='blue'))

# # Draw circles
# #circles = [ vd.Line(vd.Circle(ct, rt, c='k').orientation(nt), closed=True, lw=1.2, c='black') for ct, rt, nt in zip(ct, rt, nt) ]


# Create test mesh and center mesh 
mesh =  vd.Mesh((tv, tf), alpha = 0.8)
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

# Create hexagonal mesh
hexmesh = vd.Mesh((h_pts, dual_faces), alpha = 0.5, c=[0.2, 0.4, 0.21])

# Draw hexagonal mesh
linesHex = []

for face in dual_faces:
        hex_face = []
        for vi in face:
            hex_face.append(h_pts[vi])
        linesHex.append(vd.Line(hex_face, closed=True, lw=1.8, c='white'))
            
# Draw edges center mesh
center_edges = center.clone().wireframe().lw(1.5).flat().c('black').alpha(1)

# Draw edges Triangular mesh
edges = mesh.clone().lw(1.5).wireframe().flat().c('k').alpha(0.5)

# Draw axis lines
# Lines vd.Lines(ct- 5*nt, ct + 5*nt, c='red', lw=0.1)
clines = vd.Lines(ct[:5]- 5*nt[:5], ct[:5] + 5*nt[:5], c='tomato', lw=0.1)

vd.show(vd.Points(tv, c='red'), mesh, center, vd.Points(h_pts, c='red'), linesHex, edges, clines)
 

NameError: name 'os' is not defined

### Optimization Test

In [2]:
# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Init data files
file_pts = open(os.path.join( out_path, "points_"+str(k)+".dat"), "w")
ref = open(os.path.join( out_path, "init_directions_"+str(k)+".dat"), "w")


# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])


# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create test mesh
test_mesh =  vd.Mesh((tv, tf), alpha = 0.8)

# Draw edges Triangular mesh
edges = test_mesh.clone().lw(1.5).wireframe().flat().c('k').alpha(0.5)

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)
# Get inner vertices
inner_vertices = tmesh.inner_vertices()
# Get outer vertices
outer_vertices = tmesh.boundary_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual = tmesh.dual_top()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])


# Write data to file
for v  in tmesh.inner_vertices():
    file_pts.write(str(tmesh.vertices[v][0]) + " " + str(tmesh.vertices[v][1]) + " " + str(tmesh.vertices[v][2]) + "\n")
    ref.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")

file_pts.close()
ref.close()

# Optimization
# Init  X0 = [e_i[inner vertices]]
X = e_i[inner_vertices].flatten()

# Init constraints
linecong = LineCong()
linecong.initialize_constraint(h_pts, dual_faces, len(inner_vertices), X)

# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

for i in range(10):
    # Create file per iteration
    directions = open(os.path.join( out_path, "directions_"+str(k)+"_"+str(i)+".dat"), "w")

    # Compute J, r for each constraint
    linecong.compute(h_pts, dual_faces, X)
    
    # Add constraints  
    opt.add_constraint(linecong)

    # Optimize
    opt.optimize("LM")

    e_i = opt.X.reshape(-1,3)   

    # Write data to file
    for v  in range(len(tmesh.inner_vertices())):   
        directions.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")
    directions.close()

# Print log
opt.print_log()
e_i = opt.X.reshape(-1,3)   

# Visualize line congruence
congruence = vd.Lines(tv[0:1]- 5*e_i[0:1], tv[0:1] + 5*e_i[0:1], c=[0.8,0.5,0.5], lw=0.1)

# # Draw hexagonal mesh
linesHex = []

for face in [dual_faces[0]]:
        hex_face = []
        for vi in face:
            hex_face.append(h_pts[vi])
        linesHex.append(vd.Line(hex_face, closed=True, lw=1.8, c='white'))

# Draw hexagonal mesh
linesHex = []

for face in dual_faces:
        hex_face = []
        for vi in face:
            hex_face.append(h_pts[vi])
        linesHex.append(vd.Line(hex_face, closed=True, lw=1.8, c='white'))


# # Show the scene
# plotter.show()
#vd.show(vd.Points(tv, c='red'), test_mesh, center, vd.Points(h_pts, c='red'), linesHex, edges, congruence)

Mesh Data Structure: |V| = 469, |F| = 858, |E| = 1326
self : [[-0.40696349 -0.9126135  -0.03895278]
 [ 0.46492789 -0.88522465 -0.01481163]
 [ 0.98021468  0.19553062  0.03077279]
 [ 0.42075728  0.90655115  0.0335906 ]
 [-0.52051137  0.8537158   0.01540259]
 [-0.9991427  -0.03041845 -0.02808172]], cij = [[-0.40696349 -0.9126135  -0.03895278]
 [ 0.46492789 -0.88522465 -0.01481163]
 [ 0.98021468  0.19553062  0.03077279]
 [ 0.42075728  0.90655115  0.0335906 ]
 [-0.52051137  0.8537158   0.01540259]
 [-0.9991427  -0.03041845 -0.02808172]]
self : [[ 0.51309409  0.85533828  0.07163012]
 [-0.51603281  0.85435605 -0.06152942]
 [-0.98565223 -0.05182734 -0.16063505]
 [-0.48371037 -0.86962831 -0.0988478 ]
 [ 0.51009884 -0.85520952  0.09173798]
 [ 0.97713668 -0.14321625  0.1571401 ]], cij = [[ 0.51309409  0.85533828  0.07163012]
 [-0.51603281  0.85435605 -0.06152942]
 [-0.98565223 -0.05182734 -0.16063505]
 [-0.48371037 -0.86962831 -0.0988478 ]
 [ 0.51009884 -0.85520952  0.09173798]
 [ 0.97713668 -0.1

### Save Results

In [2]:
clines = vd.Lines(tv[:50]- 5*e_i[:50], tv[:50] + 5*e_i[:50], c='tomato', lw=0.1)

vd.show(vd.Points(tv, c='red'), mesh, center, vd.Points(h_pts, c='red'), linesHex, edges, clines)

NameError: name 'tv' is not defined

## Minimize Library test

In [3]:
from scipy import optimize


def energy(x0, h_pts, dual_faces):
     # Input
    # ct: list of vertices of central mesh
    # cf: list of faces of central mesh
    # X: variables

    # Initialize constraint \sum_{f \in F} \sum_{cj,ci \in E(f)} || e_f (cj - ci)  ||^2

    # Vector dimension
    self.ei_dim = ei_dim

    # Get number of edges per face
    edge_num = 0
    for f in range(len(cf)):
        edge_num += len(cf[f])

    self.num_edge_const = edge_num

    # Get directions
    ei = X.reshape(self.ei_dim, 3)

    # Init Jacobian and residual vector
    J = np.zeros((edge_num + len(ei), len(X)), dtype=np.float64)
    r = np.zeros( edge_num + len(ei), dtype=np.float64)

    # Compute Jacobian
    # Row index
    i = 0
    # Loop over faces
    for f in range(len(cf)):
        # Get face
        face = cf[f]

        # Get vertices
        v0 = ct[face]
        v1 = np.roll(ct[face], -1, axis=0)

        # Define direction
        cicj = (v1 - v0) / np.linalg.norm(v1 - v0, axis=1)[:, None]

        # Define Jacobian
        J[i:i + len(face), 3*f:3*f + 3] = cicj

        # Store cicj because it is a constant value
        self.cij.append(cicj)

        # Define residual
        r[i:i + len(face)] = np.dot(self.cij[f], ei[f])

        # Update row index
        i += len(face)


    # Define Jacobian for the auxiliary variable
    for f in range(len(cf)):
        J[edge_num + f, f*3:f*3+3 ] = ei[f]

    
    r[edge_num:] =np.sum ( ei*ei,  axis=1) - 1

    self.J = J
    self.r = r

### Test Circumcircle

In [18]:

# Get a list of random points
#p = np.random.rand(10,3)

p1 = np.random.rand(10,3)
p2 = np.random.rand(10,3)
p3 = np.random.rand(10,3)

c, rad, n = circle_3pts(p1,p2,p3)
print(f"centers: {c}\t rad: {rad}\t n: {n}")
circles = []
for ci in range(len(c)):
    print(f"centers: {c[ci]}\t rad: {rad[ci]}\t n: {n[ci]}")
    if ci%2 == 0:
        circles.append( vd.Circle(c[ci], r=rad[ci], c='red', alpha = 0.5).orientation(n[ci])) 
    else:
        circles.append( vd.Circle(c[ci], r=rad[ci], c="green", alpha = 0.5).orientation(n[ci])) 
pts1 = vd.Points(np.array( np.vstack((p1, p2, p3))), r = 10, c = 'black')
#pts2 = vd.Points(np.array([p1[1], p2[1], p3[1]]), r = 10, c = 'green')

vd.show(pts1,*circles)


centers: [[0.38706174 0.18167749 0.42761195]
 [0.36449233 0.53954026 0.53625345]
 [0.4790021  0.50663046 0.53923755]
 [0.72571721 0.38713089 0.48248336]
 [0.25824261 0.47411309 0.44878618]
 [0.33559659 0.53053444 0.78727609]
 [0.37826845 0.85980277 0.14845869]
 [0.43063887 0.91308919 0.36782106]
 [0.46524639 0.38363435 0.63320613]
 [0.76475174 0.3663979  0.58271725]]	 rad: [0.53952315 0.56166778 0.6054543  0.57289953 0.44627465 0.28358589
 0.43692063 0.51683852 0.5218032  0.35828794]	 n: [[-0.69549541 -0.29458511  0.65536688]
 [ 0.89194074  0.14805501 -0.42722527]
 [ 0.63249859  0.58508785 -0.50756059]
 [-0.10723602  0.63963098  0.76116532]
 [ 0.95015298 -0.29102825 -0.11185647]
 [ 0.69205191  0.40075536 -0.60038263]
 [ 0.06192396 -0.85263981 -0.51881671]
 [ 0.64050028  0.31389121 -0.70087923]
 [-0.72805365 -0.09385479  0.67906491]
 [-0.43649262 -0.83909511  0.32464379]]
centers: [0.38706174 0.18167749 0.42761195]	 rad: 0.5395231503681077	 n: [-0.69549541 -0.29458511  0.65536688]
cente

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

## Sample Planarity Opt

In [2]:
# Create four quads in 3D that merge in a vertex
v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
fcs = np.array([[0, 1, 6, 3], [2, 0, 3, 5], [7, 4, 0, 2], [4, 8, 1, 0]])

# Make mesh
m = Mesh()
m.make_mesh(v, fcs)

mesh1 = vd.Mesh((v, fcs), alpha = 0.8, c='r')

X = np.zeros((3*m.V+3*m.F), dtype=np.float64)

X[:3*m.V] = m.vertices.flatten()

# Init constraints
planarity = Planarity()
planarity.initialize_constraint(m, X)


# init optimizer
opt = Optimizer()

opt.initialize_optimizer(X)

for i in range(10):

    # Compute J, r for each constraint
    planarity.compute(m, X)
    
    # Add constraints  
    opt.add_constraint(planarity)
    opt.optimize("LM")
 

mesh2 = vd.Mesh((opt.X[:3*m.V].reshape(m.V,3),  fcs), alpha = 0.9, c='b')

print(opt.energy)

vd.show(mesh1, mesh2)



Mesh Data Structure: |V| = 9, |F| = 4, |E| = 12
[171.85628720367802, 2.036814821305436, 0.4337323303603914, 0.07705210210478547, 0.031784467050821524, 0.009450738303690037, 0.0037967153241723027, 0.0012778421481861343, 0.0004787422283860423, 0.0001682370848148311]


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [ ]:
v, f = igl.read_triangle_mesh("models/catenoid_def_1.obj")
mesh = om.read_trimesh("models/catenoid_def_1.obj")
mp.plot(v, f, c=v[:, 0])

  o catenoid_def


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0485900…

In [ ]:
def iso_stereo(n, h):
    # Isotropic model stereographic projection
    # n: normal vector
    # h: distance center to plane
    # return point in I3

    # Concatenate n[1:2] and h
    return np.vstack((n[1:2],v))/(1+ n[2])

In [ ]:
a = np.array([[1, 2, 3],[5,4,2] , [9,7,2]])
b = np.array([19,22,89])
# Add b as a column to a
c = np.column_stack((a,b))
print(a[:, 0:2])

[[1 2]
 [5 4]
 [9 7]]


In [ ]:
# Get face normals
nf = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
# Get indices v1 of each face
iv1 = f[:, 0]
# Get vertices of idv1
v1 = v[iv1, :]
# Compute dot product of each face normal with v1
h = np.sum(nf * v1, axis=1)

# Get Isotropic point per face
nv = np.column_stack((nf[:, 0:2], h)) / (1 + nf[:, 2, None])

nf = np.empty((0, 3), dtype=np.int32)

for fi in range(len(f)):
    aux = np.empty((0, ), dtype=np.int32)
    for adf in mesh.ff(mesh.face_handle(fi)):
        aux = np.hstack((aux, [adf.idx()]))
    if len(aux) == 3:
        nf = np.vstack((nf, aux))

mp.plot(nv, nf[:2])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.8548016…

In [8]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix

v, f = igl.read_triangle_mesh("models/Hall.obj")

# Draw the mesh
n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
c = np.linalg.norm(n, axis=1)
p = mp.plot(v, f, c, return_plot=True) # plot

# Compute Cuvature properties
v1, v2, k1, k2 = igl.principal_curvature(v, f)



# Compute focal surfaces
rho1 = 100
rho2 = 1/k2

step = 0.001

lamb = 0.1 * np.ones(v.shape[0], dtype=np.float64)

J = np.eye(v.shape[0], dtype=np.float64) 

# Precalculate intermediate states
vs = [v]
cs = [c]
for i in range(30):
    # Compute the gradient of the focal surface
    H = csr_matrix(J.T@J)
    b = - J.T@(lamb - rho1)
    delta = spsolve(H, b)
    lamb = lamb + step * delta
    # Update the mesh
    v = v + lamb[:,None]*n

    # Recompute the normals
    n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
    c = np.linalg.norm(n, axis=1)
    # Add to the list
    # p.update_object(vertices=v, colors=c)
    vs.append(v)
    cs.append(c)
    

# Add interactive visulization
@mp.interact(level=(0, 9))
def mcf(level=0):
    p.update_object(vertices=vs[level], colors=cs[level])
    vs.append(v)
    cs.append(c)
p

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(19.867670…

interactive(children=(IntSlider(value=0, description='level', max=9), Output()), _dom_classes=('widget-interac…